In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import cv2
import os
import random
import matplotlib.pyplot as plt
# import tensorwatch as tw

import torch
import copy
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import torch.optim.lr_scheduler as lr_scheduler

from utils.dataset import *
import torch.utils.data as data

from utils.train import *
from utils.test  import *
from utils.model_select import model_select

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Training settings
####### test作训练集 #######
# epochs = 80
# lr = 3e-6
# gamma = 0.7
# step_size = 5
###### train作训练集 #######
epochs = 100
lr = 0.008
gamma = 0.1
step_size = 30

seed = 42
device = 'cuda:1'

file_Path = '/home/a611/Projects/Datasets/NICO/'
train_name = ['/home/a611/Projects/gyc/Local_Features/csvs/NICO_animal_train_without_cage_without_dog_eating_IID_train.csv']
test_name = ['/home/a611/Projects/gyc/Local_Features/csvs/NICO_animal_train_without_cage_without_dog_eating_IID_test.csv']
num_classes = 10
num_input = 3
batch_size = 32
num_workers = 8
########################
os.chdir('examples')

In [3]:
# # from tensorboardX import SummaryWriter
# # writer = SummaryWriter('log') #建立一个保存数据用的东西
# model = torch.hub.load('facebookresearch/deit:main', 
#     'deit_tiny_patch16_224', pretrained=False)
# model.head = nn.Linear(in_features = model.head.in_features, out_features = num_classes, bias = True)
# model.to(device);

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

## Load Data

In [5]:
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

val_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)


test_transforms = transforms.Compose([transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

## Load Datasets

In [6]:
label_map = get_map(test_name)
label_key = list(label_map.keys())
train_set = MyDataset(file_Path, train_name, label_map,
                            train_transforms)
train_loader = data.DataLoader(
    dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)

valid_set = MyDataset(file_Path, test_name, label_map,
                            val_transforms)
valid_loader = data.DataLoader(
    dataset=valid_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [7]:
print(len(train_loader))
print(len(valid_loader))

291
32


## Effecient Attention

### Training

In [8]:
model = model_select('ResNet18', num_input, num_classes).to('cpu')
# print(model)
model = torch.nn.DataParallel(model)
model_state_disk = torch.load('/home/a611/Projects/gyc/Local_Features/models/test_acc_0.743311_epoch_33', map_location = 'cpu')
model.load_state_dict(model_state_disk)
model = model.module
model.linear = torch.nn.Linear(512, num_classes)

model = model.to(device)
# print(model)

# loss function
criterion = nn.CrossEntropyLoss()
# # optimizer
# optimizer = optim.Adam(model.parameters(), lr=lr)
# # scheduler
# scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9,weight_decay=5e-4)  # 优化器
decay_epoch = [30, 60, 80, 100] 
#decay_epoch = [10, 20, 30, 40]
scheduler = lr_scheduler.MultiStepLR(optimizer,
                                     milestones=decay_epoch, gamma=0.1)

In [9]:
model_ = None
highest_test_acc = 0
for i in range(epochs):
    # print('EPOCH:', i + 1)
    train_iter = iter(train_loader)
    test_iter = iter(valid_loader)
    ########################################
    train_loss, train_acc = train(model, device, train_iter, optimizer, train_set, batch_size)
    test_loss, test_acc = test(model, device, test_iter, valid_set, batch_size)
    scheduler.step()
    print( 'EPOCH: %03d, train_loss: %3f, train_acc: %3f, test_loss: %3f, test_acc: %3f'
          % (i + 1, train_loss, train_acc, test_loss, test_acc))
    if test_acc > highest_test_acc:
        highest_test_acc = test_acc
        model_ = copy.deepcopy(model)
        print('Highest test accuracy: %3f' % highest_test_acc)
        torch.save(model_, '../models/NICO_train_without_cage_without_dog_eating.model')
#     print( 'EPOCH: %03d, train_loss: %3f, train_acc: %3f' % (i + 1, train_loss, train_acc))




  0%|          | 0/291 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

EPOCH: 001, train_loss: 0.799953, train_acc: 0.788061, test_loss: 1.942819, test_acc: 0.337695
Highest test accuracy: 0.337695


  0%|          | 0/291 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

...

  0%|          | 0/291 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

EPOCH: 050, train_loss: 0.208081, train_acc: 0.935613, test_loss: 1.545610, test_acc: 0.494531


  0%|          | 0/291 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# model = torch.load('../models/CSE_train.model')
# print(model.layer4)

In [ ]:
# def draw(x, output, input_type):
#     output_max = output.argmax(dim=1)
#     output_numpy = output.cpu().detach().numpy()
#     fig, ax = plt.subplots(figsize=(12, 8))
#     y = output_numpy[0]
#     ax.bar(x = x, height = y)
#     ax.set_title('Item: %s.' % x[int(output_max)], fontsize=15);
#     xticks = ax.get_xticks()
#     for i in range(len(y)):
#         xy = (xticks[i], y[i])
#         s = '%03f' % y[i]
#         ax.annotate(
#             text=s,  # 要添加的文本
#             xy=xy,  # 将文本添加到哪个位置
#             fontsize=8,  # 标签大小
#             color="red",  # 标签颜色
#             ha="center",  # 水平对齐
#             va="baseline"  # 垂直对齐
#         )
#         plt.savefig("../results/%s_%s.jpg" % (input_type, x[int(output_max)]))
#     return output_max

# def result(x, output, input_type):
#     output_max = output.argmax(dim=1)
#     print('Input: %s; Item: %s.' % (input_type, x[int(output_max)]))
#     return output_max

In [ ]:
# model.eval()

# #########################################
# # # for input_file_name in os.listdir('inputs'):
# # input_file_name = 'doll_86.jpg'
# # image_name = os.path.join('inputs',input_file_name)
# # image = Image.open(image_name)
# # img = image.resize((224, 224))
# # input = test_transforms(image).unsqueeze(0)
# # input = input.to(device)
# # output = model(input)
# # # output_max = draw(label_key, output)
# # output_max = result(label_key, output, 'origin')
# # output_max = draw(label_key, output, 'origin')

# root_dir = '../sample_pairs'

# catagories = os.listdir(root_dir)
# catagories.sort()
# for catagory in catagories:
#     acc = 0
#     total = 0
#     print('###################### %s ######################' % catagory)
#     items = os.listdir(os.path.join(root_dir, catagory))
#     items.sort()
#     for item in items:
#         image_name = os.path.join(root_dir, catagory, item)
#         image = Image.open(image_name)
#         img = image.resize((224, 224))
#         input = test_transforms(image).unsqueeze(0)
#         input = input.to(device)
#         output, embedding = model(input)
#         # output_max = draw(label_key, output)
#         output_max = result(label_key, output, '%s, %s.' %(catagory, item))
# #         output_max = draw(label_key, output, '%s, %s.' %(catagory, item))
#         if item.split('_')[0] == label_key[int(output_max)]:
#             acc += 1
#         total += 1
#     if total:
#         print('---------------------- Acc: {}%. ----------------------'.format(format(acc/total * 100, '.2f')))